In [2]:
import threading
from pathlib import Path

import lib_ipmu_drv_config as config_drv

import lib_ipmu_daq_api as api
import lib_ipmu_daq_config as config_daq
import lib_ipmu_daq_generator as gen
import lib_ipmu_daq_aquisition as daq
import lib_ipmu_daq_process as post
import lib_ipmu_daq_graph as graph

import lib_ipmu_drv_config as config_drv
import lib_ipmu_drv_command as command

%load_ext autoreload
%autoreload 2

In [2]:
preset = config_daq._loadConfig(Path("./_config_preset.toml"))
run = config_drv._loadConfig(Path("./_config_run.toml"))
app_config = config_daq.initParams(preset, run, debug=True)


[Io]
  sample_rate         : 100000
  gen_chunk_sec       : 0.100000
  proc_interval       : 0.125000
  queue_depth         : 40
  quad_depth          : 40

[Gui]
  display_sec         : 36000
  plot_sec            : 0.150000
  gui_interval_ms     : 50
  pruning             : 100

[Logging]
  log_chunk           : 1024
  log_data_num        : 4

[Encoder Postproc]
  quadpulse_width     : 0.000250
  threshold           : 2.500000

[Debug Encoder]
  input_velocity      : 0.100000
  pulse_height        : 5.000000
  pulse_duty          : 0.500000
  pulse_phase_A       : 0.000000

[Debug Power]
  amplitude           : 0.300000
  phase               : 0.000000

[Driver]
  device_com          : COM5
  target_speed_rps    : 0.400000
  current_init        : 45
  electrical_angle_init: 200
  t_DC                : 90
  pps                 : 100
  step                : 20
  rst                 : 1
  ppsps               : 2000
  t_stablerot         : 7200
  t_excess_spindown   : 3600
  dir_rotatio

In [3]:
drv_config = config_drv.initParams(run, debug=True)


[Driver]
  device_com          : COM5
  target_speed_rps    : 0.400000
  current_init        : 45
  electrical_angle_init: 200
  t_DC                : 90
  pps                 : 100
  step                : 20
  rst                 : 1
  ppsps               : 2000
  t_stablerot         : 7200
  t_excess_spindown   : 3600
  dir_rotation        : 1
  t_current_reduction_duration: 3600
  step_current_reduction: [45, 40, 35, 30, 25, 20, 15, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1]

[Dependent]
  target_speed_rpm    : 24
  electrical_frequency: 14.400000
  pps_fin             : 14313
  t_total             : 11600.650000


In [4]:
try:
    app = api.DAQApp(config=app_config, DEBUG=False)
    app.setup() # initialize widgets
    app.initStorer(runs_dir=config_daq.RUNS_DIR) # initialize HDF5 storage
    app.initLogger()  # Initialize logger in the main thread to avoid issues


    # generater for debug
    # generator = gen.Generator(config=app_config, buf_q=app.buf_q, stop_event=app.stop_event)
    
    # Motor
    motor = command.Command(config=drv_config, comvel_q=app.comvel_q, stop_event=app.stop_event)
    motor._reset()
    motor.setup(DEBUG=True)

    # acquisiton for measurements
    acquisition = daq.DataAquisition(config=app_config, buf_q=app.buf_q, stop_event=app.stop_event)

    # processor
    processor = post.Processor(config=app_config, buf_q=app.buf_q, quad_q=app.quad_q, comvel_q=app.comvel_q, DataStoreFlag = motor.DataStoreFlag, stop_event=app.stop_event, h5f=app.h5f, dset=app.dset, debug=app.DEBUG, logger=app.logger)

    # Graph
    gui = graph.DAQGUI(config=app_config, quad_q=app.quad_q, stop_event=app.stop_event)

    # 5. Create the thread objects individually
    #logger_thread = threading.Thread(target=app.initLogger, daemon=True)
    #generator_thread = threading.Thread(target=generator.run, daemon=True)
    acquisition_thread = threading.Thread(target=acquisition.run, daemon=True)
    processor_thread = threading.Thread(target=processor.run, daemon=True)
    motor_thread = threading.Thread(target=motor.run, daemon=True)

    # 6. Start the threads (now controlled from the top level)
    #logger_thread.start()
    #generator_thread.start()
    acquisition_thread.start()
    processor_thread.start()
    motor_thread.start()
    
    gui.run()

finally:
    app.shutdown()
    #logger_thread.join(timeout=0.1)
    #generator_thread.join(timeout=0.1)
    
    acquisition_thread.join(timeout=0.1)
    processor_thread.join(timeout=0.1)


HDF5 dataset created at: C:\Users\IPMU\Desktop\ipmu_DAQ\src\..\runs\251105183150.h5
Logger setup skipped: DEBUG mode is OFF.
Driver has been shutdowned properly (o_ _)o
Rotational frequency                           :  0.4 [Hz]
Current pulse per second to finish acceleration:  14313
Initial current                                :  450.000000 [mA]
Initial Pulse per second                       :  100
Maximum acceleration step                      :  2000
Rotational shift after every pps               :  1 [s]
Step                                           :  20
Stable rotation duration                       :  120.0 [min]
excess time before stop the rotor command      :  60.0 [min]
Total rotation duration time                   :  193.34416666666667 [min]
All process start time 2025-11-05 18:31:51.563156 (*ゝ∀･)v
operation setting: 04010409
pulse setting: 04010409
initial position: 200, status: 04010409
High-rate speed setting: 100 [pps], status: 04010409
06012264008d
startup speed sett

In [ ]:
# config
preset = config_daq._loadConfig(Path("./_config_preset.toml"))
run = config_drv._loadConfig(Path("./_config_run.toml"))

# initialize
app_config = config_daq.initParams(preset, run, debug=True)
drv_config = config_drv.initParams(run, debug=True)

# setup
app = api.DAQApp(config=app_config, DEBUG=False)
app.setup() # initialize widgets

# motor setup
motor = command.Command(config=drv_config, comvel_q=app.comvel_q, stop_event=app.stop_event)

# reset
motor._reset()



[Io]
  sample_rate         : 100000
  gen_chunk_sec       : 0.100000
  proc_interval       : 0.125000
  queue_depth         : 40
  quad_depth          : 40

[Gui]
  display_sec         : 36000
  plot_sec            : 0.150000
  gui_interval_ms     : 50
  pruning             : 100

[Logging]
  log_chunk           : 1024
  log_data_num        : 4

[Encoder Postproc]
  quadpulse_width     : 0.000250
  threshold           : 2.500000

[Debug Encoder]
  input_velocity      : 0.100000
  pulse_height        : 5.000000
  pulse_duty          : 0.500000
  pulse_phase_A       : 0.000000

[Debug Power]
  amplitude           : 0.300000
  phase               : 0.000000

[Driver]
  device_com          : COM5
  target_speed_rps    : 0.200000
  current_init        : 45
  electrical_angle_init: 200
  t_DC                : 90
  pps                 : 100
  step                : 20
  rst                 : 1
  ppsps               : 2000
  t_stablerot         : 7200
  t_excess_spindown   : 3600
  dir_rotatio